In [3]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [4]:
main_path = r"C:\Users\SEBASTIAN\OneDrive - Universidad de los Andes\OctavoSemestre\ProblemaEspecial\ML/dataSets/"
pd_data_v1 = pd.read_csv(main_path+'dataSet_dron_v1.csv')
pd_data_v4 = pd.read_csv(main_path + 'dataSet_dron_v8_total_salo.csv')

main_path = r"C:\Users\SEBASTIAN\OneDrive - Universidad de los Andes\OctavoSemestre\ProblemaEspecial\ML/paquetes/v8 y v9/"
path_energy = main_path + "featuresEnergy_v8.csv"
path_energy_v2 = main_path + "featuresEnergy_v9.csv"
path_turns = main_path + "distanceTurn_v8.csv"
path_turns_v2 = main_path + "distanceTurn_v9.csv"

data_energy = pd.read_csv(path_energy)
data_energy_v2 = pd.read_csv(path_energy_v2)

pd_data_v4 = pd.concat([data_energy,data_energy_v2])
pd_data = pd_data_v4

pd_data = pd_data_v4

### Read the data

In [5]:
X = pd_data.drop(columns=["missing_points","path_num","sim_drone_time","Energy"],axis=1)
y = pd_data["Energy"]

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=35)

X_train_mean = x_train.mean(axis=0)
X_train_std = x_train.std(axis=0)

X_train_mean_array = X_train_mean.to_numpy()
X_train_std_array = X_train_std.to_numpy()
x_train_array = x_train.to_numpy()
x_test_array = x_test.to_numpy()

x_train_array = (x_train_array-X_train_mean_array)/X_train_std_array
x_test_array = (x_test_array-X_train_mean_array)/X_train_std_array
y_train_array = y_train.to_numpy()

columns = x_train.columns
x_train_v1 = pd.DataFrame(data=x_train_array,columns=columns)
x_test_v1 = pd.DataFrame(data=x_test_array,columns=columns)

In [7]:
print(f'Train size/Test size {len(x_train),len(x_test)}')

Train size/Test size (940, 235)


### Model - Neural network

In [8]:
NN_model = Sequential()

NN_model.add(Dense(128,kernel_initializer='normal',input_dim=6,activation='relu'))

NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

NN_model.compile(loss="mean_squared_logarithmic_error", optimizer=Adam(learning_rate=0.01), metrics=["mean_squared_logarithmic_error"])
NN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               896       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 149,249
Trainable params: 149,249
Non-trainable params: 0
__________________________________________________

### Checkpoint

In [9]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5'
path_weigths = r"C:\Users\SEBASTIAN\OneDrive - Universidad de los Andes\OctavoSemestre\ProblemaEspecial\ML\ModelosNN2/" 
final_path = path_weigths+checkpoint_name
checkpoint = ModelCheckpoint(final_path, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
reduceLR = ReduceLROnPlateau()
callbacks_list = [checkpoint,reduceLR]

In [10]:
NN_model.fit(x_train_array, y_train_array, epochs=500, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Epoch 1/500
7/7 [==============================] - 2s 60ms/step - loss: 262.7455 - mean_squared_logarithmic_error: 262.7455 - val_loss: 104.4112 - val_mean_squared_logarithmic_error: 104.4112

Epoch 00001: val_loss improved from inf to 104.41121, saving model to C:\Users\SEBASTIAN\OneDrive - Universidad de los Andes\OctavoSemestre\ProblemaEspecial\ML\ModelosNN2\Weights-001--104.41121.hdf5
Epoch 2/500
7/7 [==============================] - 0s 17ms/step - loss: 83.8198 - mean_squared_logarithmic_error: 83.8198 - val_loss: 45.6645 - val_mean_squared_logarithmic_error: 45.6645

Epoch 00002: val_loss improved from 104.41121 to 45.66451, saving model to C:\Users\SEBASTIAN\OneDrive - Universidad de los Andes\OctavoSemestre\ProblemaEspecial\ML\ModelosNN2\Weights-002--45.66451.hdf5
Epoch 3/500
7/7 [==============================] - 0s 17ms/step - loss: 37.8214 - mean_squared_logarithmic_error: 37.8214 - val_loss: 25.8492 - val_mean_squared_logarithmic_error: 25.8492

Epoch 00003: val_loss impro

In [11]:
y_predict = NN_model.predict(x_test_array)
r2 = r2_score(y_test,y_predict)
RMSE = mean_squared_error(y_test,y_predict,squared=False)
MSE = mean_squared_error(y_test,y_predict)

print(f'r2: {r2}')
print(f'RMSE: {RMSE}')
print(f'MSE: {MSE}')

r2: 0.10800829776090137
RMSE: 4103631931.465499
MSE: 1.6839795028943262e+19


In [15]:
y_predict = NN_model.predict(x_test_array)
r2 = r2_score(y_test,y_predict)
RMSE = mean_squared_error(y_test,y_predict,squared=False)
MSE = mean_squared_error(y_test,y_predict)

print(f'r2: {r2}')
print(f'RMSE: {RMSE}')
print(f'MSE: {MSE}')

r2: -0.0757046862113544
RMSE: 4506449655.414875
MSE: 2.0308088496788845e+19


In [20]:
y_predict = NN_model.predict(x_test_array)
r2 = r2_score(y_test,y_predict)
RMSE = mean_squared_error(y_test,y_predict,squared=False)
MSE = mean_squared_error(y_test,y_predict)

print(f'r2: {r2}')
print(f'RMSE: {RMSE}')
print(f'MSE: {MSE}')

r2: 0.10344388930645987
RMSE: 4114117876.4038863
MSE: 1.6925965900946022e+19


In [11]:

y_predict = NN_model.predict(x_test_array)
r2 = r2_score(y_test,y_predict)
RMSE = mean_squared_error(y_test,y_predict,squared=False)
MSE = mean_squared_error(y_test,y_predict)

print(f'r2: {r2}')
print(f'RMSE: {RMSE}')
print(f'MSE: {MSE}')

r2: -0.18687783743360753
RMSE: 62118865.23510145
MSE: 3858753418096695.5
